In [1]:
import pandas as pd
import json
import numpy as np

In [6]:
DATA_DIR = '../data/'
TWEET_DIR = '../data/tweets/'
NUM_CLUSTERS = 6
events = open(DATA_DIR + 'event_names.txt', 'r').read().splitlines()

In [3]:
data_filter_method = '_noRT' # make sure to include underscore
cluster_method = ''

In [4]:
time_dict = json.load(open(DATA_DIR + "event_year.json","r"))
polarization_dict = json.load(open(DATA_DIR + "polarization"+data_filter_method+".json","r"))

In [14]:
between_topic_pol = json.load(open(DATA_DIR + "between_topic_polarization"+cluster_method+".json","r"))

## plotting overall polarization

In [9]:
x = []
y = []
y_random = []
labels=[]
ex = ['fort_lauderdale']  # we exclude Fort Lauderdale because we only have data for the first day after the shooting
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    x_val = float(2000+t)
    y_val = float(polarization_dict[e][0])
    y_random_val = float(float(polarization_dict[e][1]))
    x.append(x_val)
    y.append(y_val)
    y_random.append(y_random_val)
    labels.append(new_label)
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y + y_random), 'label':labels * 2, 'is_actual':['actual value']* len(y) + ['value resulting from random party assignment']* len(y)})
df.to_csv(DATA_DIR + "polarization"+data_filter_method+".csv")


## plotting within vs between topic polarization

In [7]:
topics = []
for event in events:
    labels = np.load(TWEET_DIR + event + '/' + event + '_cluster_labels_' + str(NUM_CLUSTERS) + cluster_method + '.npy')
    topics.extend(list(labels))
topics = np.array(topics)

In [ ]:
x = []
y_between = []
y_within = []
labels=[]
race = []
ex = ['fort_lauderdale']
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    within_topic_pol = json.load(open(TWEET_DIR +e+"/"+e+"_topic_polarization"++".json","r"))
    
    # get the topic proportions
    data = pd.read_csv('all_events/' + e + '/' + e + '.csv', sep='\t', lineterminator='\n',usecols=['remove'])
    indices = np.load('all_events/' + e + '/' + e + '_cleaned_indices_partisan.npy')
    strict_indices = np.load('all_events/' + e + '/' + e + '_comp_cluster_indices.npy')
    strict_labels = np.load('all_events/' + e + '/' + e + '_comp_cluster_labels.npy')
    data = data.iloc[indices]
    print(len(data))
    data.reset_index(drop=True, inplace=True)
    data = data.iloc[strict_indices]
    data['cluster'] = strict_labels
    data = data[~data['remove']]
    topics = np.array(data['cluster'])
    
    
    #topics = np.load('all_events/' + e + '/' + e + '_cluster_labels_'+str(num_clusters)+'.npy')
    within_topic_pol_val = (np.array([float(within_topic_pol[str(i)][0]) for i in range(num_clusters)]) * np.bincount(topics)).sum() / len(topics)
    
    x_val = float(2000+t)
    y_between_val = float(between_topic_pol[e][0])
    y_within_val = within_topic_pol_val
    x.append(x_val)
    y_between.append(y_between_val)
    y_within.append(y_within_val)
    labels.append(new_label)
    race.append(shooter_race[e])
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y_between + y_within), 'label':labels * 2, 'kind':['between-topic']* len(y_between) + ['within-topic']* len(y_within), 'race':race * 2})
df.to_csv('all_events/strict_topic_polarization_naive_bayes.csv', index=False)